In [8]:
import chess
import chess.engine
from statistics import mean

# Function to get multiple best moves from Maia using multipv
def get_multiple_best_moves(engine, board, n=3):
    result = engine.analyse(board, chess.engine.Limit(nodes=1000), multipv=n)
    best_moves = [(info["pv"][0], info["score"].relative.score(mate_score=10000)) for info in result]
    # Sort the best moves based on their evaluation scores
    best_moves_sorted = sorted(best_moves, key=lambda x: x[1], reverse=True)
    return best_moves_sorted

# Function to evaluate Maia easiness
def evaluate_maia_easiness(fen, best_move, maia_engines, multipv=3):
    board = chess.Board(fen)
    easiness_scores = {}
    print(f"Evaluating Maia easiness for FEN: {fen}, Best Move: {best_move}")

    for elo, engine in maia_engines.items():
        multiple_best_moves = get_multiple_best_moves(engine, board, multipv)
        print(f"Maia {elo} best moves: {multiple_best_moves}")

        move_probabilities = {move: (i + 1) / len(multiple_best_moves) for i, (move, score) in enumerate(multiple_best_moves)}

        if best_move in move_probabilities:
            easiness_score = move_probabilities[best_move]
        else:
            easiness_score = 1.0  # Assign 1.0 if the best move is not in the PV list

        easiness_scores[elo] = easiness_score

    # Combine easiness scores from different models into a single score
    combined_easiness_score = mean(easiness_scores.values()) if easiness_scores else 1.0
    return combined_easiness_score

# Example usage:
fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
best_move = chess.Move.from_uci("e2e4")

maia_engines = {
    1100: chess.engine.SimpleEngine.popen_uci(["lc0", f"--weights=maia_weights/maia-1100.pb.gz"]),
    1200: chess.engine.SimpleEngine.popen_uci(["lc0", f"--weights=maia_weights/maia-1200.pb.gz"]),
    # Add other Maia engines as needed
}

easiness_scores = evaluate_maia_easiness(fen, best_move, maia_engines)
print(f"Easiness scores: {easiness_scores}")

# Cleanup Maia engines
for engine in maia_engines.values():
    engine.quit()


<UciProtocol (pid=19865)>: stderr >>        _
<UciProtocol (pid=19865)>: stderr >> |   _ | |
<UciProtocol (pid=19865)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=19866)>: stderr >>        _
<UciProtocol (pid=19866)>: stderr >> |   _ | |
<UciProtocol (pid=19866)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=19865)>: stderr >> Loading weights file from: maia_weights/maia-1100.pb.gz
<UciProtocol (pid=19865)>: stderr >> Creating backend [metal]...
<UciProtocol (pid=19865)>: stderr >> Initialized metal backend on device Apple M1 Pro


Evaluating Maia easiness for FEN: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1, Best Move: e2e4


<UciProtocol (pid=19866)>: stderr >> Loading weights file from: maia_weights/maia-1200.pb.gz
<UciProtocol (pid=19866)>: stderr >> Creating backend [metal]...
<UciProtocol (pid=19866)>: stderr >> Initialized metal backend on device Apple M1 Pro


Maia 1100 best moves: [(Move.from_uci('d2d4'), 132), (Move.from_uci('c2c4'), 129), (Move.from_uci('e2e4'), 99)]
Maia 1200 best moves: [(Move.from_uci('c2c4'), 134), (Move.from_uci('d2d4'), 124), (Move.from_uci('e2e4'), 86)]
Easiness scores: 1.0
